In [1]:
%load_ext autoreload
%autoreload 2

In [98]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd
import ipyvuetify as v
import ipywidgets as w
from plotly import graph_objects as go

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.fields import fields_reference
from api.data.processing import post_process
from api.data.base import tables, data_path
from api.data.utils import _years, _capital_stack
import utils

timestamp = f"{pd.Timestamp.today():%Y%m%d_%H%M%S}"

root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

### Bloomberg Connection

cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

cds_ref = pd.read_csv(data_path / 'cds_reference.csv').query('exists == True').drop_duplicates(subset='security')



cds_data = cds_hist.merge(cds_ref, on='security', how='left') \
        .drop(['exists', 'ticker', 'cds_ticker_5y'], axis=1) \
        .dropna(subset='last_price')

issuer = 'ISP IM Equity'
years = 5
i = 1
cds_pairs = [
    'SLA / SR',
    'SUB / SLA',
    'SUB / SR'
]

issuer_sel = v.Autocomplete(
    v_model=None,
    items=cds_ref['issuer_equity_ticker'].drop_duplicates().to_list(),
    label='issuer',
    dense=True,
    outlined=True
)

year_sel = v.Autocomplete(
    v_model=None,
    items=_years,
    label='years',
    dense=True,
    outlined=True
)

cap_stack_sel = v.Autocomplete(
    v_model=None,
    items=cds_pairs,
    label='capital stack',
    dense=True,
    outlined=True
)

chart_btn = v.Btn(
    left=True,
    children=[
        v.Icon(children=['mdi-chart-line']),
        'Chart'
    ],
)

param_box = v.Container(
    children=[
        v.Row(
            children=[
                v.Col(
                    cols=2,
                    children=[issuer_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[year_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[cap_stack_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[chart_btn],
                    class_="my-0 py-0"
                ),
            ],
            align_content='center',
        ),
    ]
)
out = w.Output()
tab = w.VBox(
    children=[
        param_box,
        out,
    ]
)

tab

@out.capture(clear_output=True)
def on_click_plot(widget, event, payload):
    issuer = issuer_sel.v_model
    years = year_sel.v_model
    cds_data_ = cds_data.query(f'issuer_equity_ticker == "{issuer}" and years == {years}')
    pair = cap_stack_sel.v_model.split(' / ')
    s1 = cds_data_.query(f'capital_stack == "{pair[0]}"').set_index('date')['last_price']
    s2 = cds_data_.query(f'capital_stack == "{pair[1]}"').set_index('date')['last_price']
    
    ratio = (s1 / s2).dropna().rename(cap_stack_sel.v_model)
    fig = ratio.plot(backend='plotly', template='plotly_white')
    display(fig)
    

In [99]:
chart_btn.on_event('click', on_click_plot)

In [61]:
cds_data_ = cds_data.query(f'issuer_equity_ticker == "{issuer}" and years == {years}')

In [63]:
res = []
for pair in cds_pairs:
    s1 = cds_data_.query(f'capital_stack == "{pair[0]}"').set_index('date')['last_price']
    s2 = cds_data_.query(f'capital_stack == "{pair[1]}"').set_index('date')['last_price']
    res.append((s1 / s2).dropna().rename(' / '.join(pair)))

In [69]:
res[i].plot(backend='plotly', template='plotly_white')

In [66]:
pd.concat(res, axis=1).plot(backend='plotly')

In [36]:
s1

date
2020-09-03    112.5
2020-09-04    111.5
2020-09-07    112.5
2020-09-08    115.5
2020-09-09    113.5
              ...  
2022-06-02    235.5
2022-06-03    235.5
2022-06-06    235.5
2022-06-07    235.5
2022-06-08    232.5
Name: last_price, Length: 1294, dtype: float64

In [ ]:
cds_5y = (bond_ref['cds_ticker_5y']).dropna().drop_duplicates().to_list()

In [ ]:
field = 'SECURITY_DES'
res = bq.bdp(
    securities=cds_5y,
    fields=[field]
)

In [ ]:
from itertools import product
years = [2, 5, 7, 10]
capital_stack = ['SR', 'SUB', 'SLA']

def get_cds_generic_name(security):
    field = 'CDS_SPREAD_TICKER_5Y'
    cds_generic_ticker = bq.bdp(
        securities=[security],    
        fields=[field]
    )
    cds_generic_ticker = cds_generic_ticker[field].squeeze() + ' Curncy'

    field = 'SECURITY_DES'
    cds_generic_name = bq.bdp(
        securities=[cds_generic_ticker],    
        fields=[field]
    )
    cds_generic_name = cds_generic_name[field].squeeze()
    return cds_generic_name

def get_cds_tickers(
    security: str = None,
    cds_generic_name=None,
    bq=None
):
    
    if security and not cds_generic_name:
        cds_generic_name = get_cds_generic_name(security)
    cds_generic_name = ' '.join(cds_generic_name.split(' ', 3)[:-1])
    tickers = []
    for yrs, cs in product(years, capital_stack):
        cds_ticker = f"{cds_generic_name} {cs} {yrs}Y D14 Curncy"
        tickers.append({'name': cds_ticker , 'years': yrs, 'capital_stack': cs})
    return pd.DataFrame(tickers)

In [ ]:
cds_ref = []
for index, row in res.iterrows():
    cds_ticker_5y, cds_generic_name = row['security'], row[field]
    cds_tickers = get_cds_tickers(cds_generic_name=cds_generic_name)
    cds_tickers['cds_ticker_5y'] = cds_ticker_5y
    cds_ref.append(cds_tickers)
cds_ref = pd.concat(cds_ref)

In [ ]:
to_merge = bond_ref[['ticker', 'issuer_equity_ticker', 'cds_ticker_5y']].dropna().drop_duplicates(subset='cds_ticker_5y').set_index('cds_ticker_5y')

In [ ]:
cds_ref = cds_ref.merge(to_merge, how='left', left_on='cds_ticker_5y', right_index=True)

In [ ]:
cds_ref.to_csv(data_path / 'cds_reference.csv', index=False)

In [ ]:
cds_prices = [] 
cds_na = []

for sec in cds_ref['name']:
    try:
        res = bq.bdp(
            securities=[sec],
            fields=['PX_LAST']
        )
        print(f'{sec} ---> OK')
        cds_prices.append(res)
    except TypeError:
        cds_na.append(sec)
        print(f'{sec} ---> NA')

BBVA CDS EUR SR 2Y D14 Curncy ---> OK
BBVA CDS EUR SUB 2Y D14 Curncy ---> OK
BBVA CDS EUR SLA 2Y D14 Curncy ---> OK
BBVA CDS EUR SR 5Y D14 Curncy ---> OK
BBVA CDS EUR SUB 5Y D14 Curncy ---> OK
BBVA CDS EUR SLA 5Y D14 Curncy ---> OK
BBVA CDS EUR SR 7Y D14 Curncy ---> OK
BBVA CDS EUR SUB 7Y D14 Curncy ---> OK
BBVA CDS EUR SLA 7Y D14 Curncy ---> OK
BBVA CDS EUR SR 10Y D14 Curncy ---> OK
BBVA CDS EUR SUB 10Y D14 Curncy ---> OK
BBVA CDS EUR SLA 10Y D14 Curncy ---> OK
DANBNK CDS EUR SR 2Y D14 Curncy ---> OK
DANBNK CDS EUR SUB 2Y D14 Curncy ---> OK
DANBNK CDS EUR SLA 2Y D14 Curncy ---> OK
DANBNK CDS EUR SR 5Y D14 Curncy ---> OK
DANBNK CDS EUR SUB 5Y D14 Curncy ---> OK
DANBNK CDS EUR SLA 5Y D14 Curncy ---> OK
DANBNK CDS EUR SR 7Y D14 Curncy ---> OK
DANBNK CDS EUR SUB 7Y D14 Curncy ---> OK
DANBNK CDS EUR SLA 7Y D14 Curncy ---> OK
DANBNK CDS EUR SR 10Y D14 Curncy ---> OK
DANBNK CDS EUR SUB 10Y D14 Curncy ---> OK
DANBNK CDS EUR SLA 10Y D14 Curncy ---> OK
AIB CDS EUR SR 2Y D14 Curncy ---> OK
AIB C

In [ ]:
cds_prices = pd.concat(cds_prices)

In [ ]:
cds_ref['exists'] = ~cds_ref['name'].isin(cds_na)

In [ ]:
cds_ref.to_csv(data_path / 'cds_reference.csv', index=False)

In [ ]:
cds_prices.to_csv(data_path / 'cds_market.csv', index=False)